In [1]:
from typing import List, Tuple
import pandas as pd
import datetime
import requests
import os

In [2]:
def get_assets(spolka_excel: pd.DataFrame) -> pd.DataFrame:
    assets_line = 29

    non_current_assets_start = 31
    non_current_assets_stop = 43

    current_assets_start = 44
    current_assets_stop = 55

    sale_and_discounting = 55

    called_up_capital = 56

    own_shares = 57

    return pd.concat([
        cut_rows(spolka_excel, assets_line),
        cut_rows(spolka_excel, non_current_assets_start, non_current_assets_stop),
        cut_rows(spolka_excel, current_assets_start, current_assets_stop),
        cut_rows(spolka_excel, sale_and_discounting),
        cut_rows(spolka_excel, called_up_capital),
        cut_rows(spolka_excel, own_shares)
    ], ignore_index=True)

In [3]:
def get_eq_and_liab(spolka_excel: pd.DataFrame) -> pd.DataFrame:
    eq_and_liab_line = 59

    esotp_start = 61
    esotp_stop = 68

    nc_i_line = 68

    ncl_start = 70
    ncl_stop = 80

    cl_start = 81
    cl_stop = 91

    lrtahfsado_line = 91

    return pd.concat([
        cut_rows(spolka_excel, eq_and_liab_line),
        cut_rows(spolka_excel, esotp_start, esotp_stop),
        cut_rows(spolka_excel, nc_i_line),
        cut_rows(spolka_excel, ncl_start, ncl_stop),
        cut_rows(spolka_excel, cl_start, cl_stop),
        cut_rows(spolka_excel, lrtahfsado_line)
    ], ignore_index=True)

In [4]:
def get_years(spolka_excel: pd.DataFrame) -> pd.DataFrame:
    year_line = 28
    sl = spolka_excel.iloc[year_line:year_line+1, 2:].reset_index(drop=True)
    sl.columns = range(sl.shape[1])
    sl[0] = 'YEAR'

    return sl.applymap(lambda x: int(x[0:4]) if x != 'YEAR' else 'YEAR')

In [5]:
def cut_rows(stock_df: pd.DataFrame, start_row: int, stop_row=None) -> pd.DataFrame:
    if stop_row is None:
        rows = stock_df.iloc[start_row:start_row+1, 2:].reset_index(drop=True)
    else:
        rows = stock_df.iloc[start_row:stop_row, 2:].reset_index(drop=True)
    rows.columns = range(rows.shape[1])
    return rows.fillna(0)

In [6]:
def get_last_price_from_bankier(stock_name: str, from_year: int) -> float:
    date_from = datetime.datetime.fromisoformat(f'{str(from_year)}-12-10T00:00:00+00:00').strftime('%s')
    date_to = datetime.datetime.fromisoformat(f'{str(from_year)}-12-31T23:59:59+00:00').strftime('%s')
    response = requests.get(f'https://www.bankier.pl/new-charts/get-data?date_from={date_from}000&date_to={date_to}000&symbol={stock_name}&intraday=false&type=area')
    return response.json()['main'][-1][-1]

In [7]:
def prepare_full_info_df(spolka_excel, spolka_name, category):
    print(f'category: {category}, stock: {spolka_name}')
    assets = get_assets(spolka_excel)[list(range(1, spolka_excel.shape[1]-2))]
    assets.columns = range(assets.shape[1])
    
    all_assets = list(assets.T[0]) # full assets value
    
    assets_parts = assets.iloc[1:].reset_index(drop=True)
    assets_parts.columns = range(assets_parts.shape[1])
    
    eq_and_liab = get_eq_and_liab(spolka_excel)[list(range(1, spolka_excel.shape[1]-2))]
    eq_and_liab.columns = range(eq_and_liab.shape[1])
    
    all_eq_and_liab = list(eq_and_liab.T[0])
    
    eq_and_liab_parts = eq_and_liab.iloc[1:].reset_index(drop=True)
    eq_and_liab_parts.columns = range(eq_and_liab_parts.shape[1])

    years = list(get_years(spolka_excel).T[1:][0])
    
    issued_shares = list(get_issued_shares(spolka_excel).T[0])

    
    prices = []

    info_as_dict = {
        'stock_category': [],
        'company': [],
        'year': [],
        'assets': [],
        'assets_change_in_year': [],
        'eq_and_liab': [],
        'eq_and_liab_change_in_year': [],
        'stock_value': [],
        'year_value_change': []
    }
    
    for index, year in enumerate(years):
        try:
            prices.append(get_last_price_from_bankier(spolka_name, year))
        except:
            print(f'Error occured in: {spolka_name}, year: {year}')
            prices.append(-1)
            years[index] = -1
            continue

        if len(info_as_dict['assets_change_in_year']) != 0:
            assets_change_in_year = list(pd.DataFrame(assets_parts[index] / assets_parts[index-1] - 1).fillna(0.0).iloc[:,0])
        else:
            assets_change_in_year = [0.0] * assets_parts.shape[0]

        if len(info_as_dict['eq_and_liab_change_in_year']) != 0:
            eq_and_liab_change_in_year = list(pd.DataFrame(eq_and_liab_parts[index] / eq_and_liab_parts[index-1] - 1).fillna(0.0).iloc[:,0])
        else:
            eq_and_liab_change_in_year = [0.0] * eq_and_liab_parts.shape[0]

        info_as_dict['stock_category'].append(category)
        info_as_dict['company'].append(spolka_name)
        info_as_dict['year'].append(years[index])
        info_as_dict['assets'].append(percentage_list(all_assets[index], assets_parts[index]))
        info_as_dict['assets_change_in_year'].append(assets_change_in_year)
        info_as_dict['eq_and_liab'].append(percentage_list(all_eq_and_liab[index], eq_and_liab_parts[index]))
        info_as_dict['eq_and_liab_change_in_year'].append(eq_and_liab_change_in_year)
        info_as_dict['stock_value'].append(round(issued_shares[index] * prices[index], 2))
        info_as_dict['year_value_change'].append((prices[index]/prices[index-1]) - 1.0 if len(info_as_dict['year_value_change']) != 0 else 0.0)
        
    return info_as_dict

In [8]:
def get_assets_names_with_id(spolka_excel) -> List[Tuple[int, str]]:
    return list(enumerate(list(get_assets(spolka_excel)[0][1:])))

In [9]:
def get_eq_and_liab_names_with_id(spolka_excel) -> List[Tuple[int, str]]:
    return list(enumerate(list(get_eq_and_liab(spolka_excel)[0][1:])))

In [10]:
def get_issued_shares(spolka_excel):
    return cut_rows(spolka_excel, 18).iloc[0:1, 1:].reset_index(drop=True) * 1000

In [11]:
def percentage_list(full_value, list_of_values):
    return list(list_of_values/full_value)

In [12]:
def stock_prepared_category(category, category_list):
    xlsxs = list(filter(lambda x: not x.startswith('~'), os.listdir(f'./spolki/{category}')))
    xlsxs.sort()

    for xlsx in xlsxs:
        stock_df = pd.read_excel(f'spolki/{category}/{xlsx}', 'YS', header=None).dropna('columns', 'all')
        stock_name = xlsx.split('.')[0] if not xlsx.startswith('OPONEO.PL') else 'OPONEO.PL' # oponeo is exception in names
        category_list.append(pd.DataFrame.from_dict(prepare_full_info_df(stock_df, stock_name, category)))

In [13]:
def print_wrong_stock_names(list_of_categories):
    for category in list_of_categories:
        names = list(map(lambda x: x.split('.')[0] if not x.startswith('OPONEO.PL') else 'OPONEO.PL', list(filter(lambda x: not x.startswith('~'), os.listdir(f'./spolki/{category}')))))
        names.sort()
        print()
        print(f'Category: {category}')
        for name in names:
            date_from = datetime.datetime.fromisoformat(f'{str(1990)}-12-10T00:00:00+00:00').strftime('%s')
            date_to = datetime.datetime.fromisoformat(f'{str(1990)}-12-31T23:59:59+00:00').strftime('%s')
            response = requests.get(f'https://www.bankier.pl/new-charts/get-data?date_from={date_from}000&date_to={date_to}000&symbol={name}&intraday=false&type=area')

            if response.status_code == 404:
                print(name)

In [14]:
cat = {category: [] for category in range(2, 9)}

In [15]:
for category in range(2, 9):
    stock_prepared_category(category, cat[category])

category: 2, stock: BEDZIN
Error occured in: BEDZIN, year: 1997
category: 2, stock: ENEA
Error occured in: ENEA, year: 2002
Error occured in: ENEA, year: 2003
Error occured in: ENEA, year: 2004
Error occured in: ENEA, year: 2005
Error occured in: ENEA, year: 2007
category: 2, stock: ENERGA
Error occured in: ENERGA, year: 2011
Error occured in: ENERGA, year: 2012
category: 2, stock: KOGENERA
Error occured in: KOGENERA, year: 1997
Error occured in: KOGENERA, year: 1998
Error occured in: KOGENERA, year: 1999
category: 2, stock: LOTOS
Error occured in: LOTOS, year: 2001
Error occured in: LOTOS, year: 2002
Error occured in: LOTOS, year: 2003
Error occured in: LOTOS, year: 2004
category: 2, stock: MLSYSTEM
Error occured in: MLSYSTEM, year: 2017
category: 2, stock: PEP
Error occured in: PEP, year: 2001
Error occured in: PEP, year: 2002
Error occured in: PEP, year: 2003
Error occured in: PEP, year: 2004
category: 2, stock: PETROLINV
Error occured in: PETROLINV, year: 2006
category: 2, stock: P

In [16]:
all_categories = pd.concat([
    pd.concat(cat[category], ignore_index=True) for category in range(2, 9)
], ignore_index=True)

In [17]:
all_categories.to_csv('cleaned_and_prepared_stocks.csv', sep=';')


In [63]:
orlen_excel = pd.read_excel(f'spolki/2/PKNORLEN.xlsx', 'YS', header=None).dropna('columns', 'all')
assets_names_with_id = get_assets_names_with_id(orlen_excel)
eq_and_liab_names_with_id = get_eq_and_liab_names_with_id(orlen_excel)

eq_and_assets_names_with_ids_df = {
    'type': [],
    'id': [],
    'name': []
}
for asset in assets_names_with_id:
    eq_and_assets_names_with_ids_df['type'].append('asset')
    eq_and_assets_names_with_ids_df['id'].append(asset[0])
    eq_and_assets_names_with_ids_df['name'].append(asset[1])

for eq_and_liab in eq_and_liab_names_with_id:
    eq_and_assets_names_with_ids_df['type'].append('eq_and_liab')
    eq_and_assets_names_with_ids_df['id'].append(eq_and_liab[0])
    eq_and_assets_names_with_ids_df['name'].append(eq_and_liab[1])

pd.DataFrame.from_dict(eq_and_assets_names_with_ids_df).to_csv('assets_and_eq_and_liab_ids.csv', sep=';')